### Exporting keys

In [29]:
import configparser
import os

In [30]:
config_file = 'keys.cfg'

In [31]:
config = configparser.ConfigParser()
config.read(config_file)

['keys.cfg']

In [130]:
os.environ['SPOTIPY_CLIENT_ID'] = config['SPOTIFY']['CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = config['SPOTIFY']['CLIENT_SECRET']
# os.environ['SPOTIPY_REDIRECT_URI'] = 'https://ca416ba83b7c.ngrok.io'

### Spotify searching

#### Workaround for getting an OAuth 2.0 access and refresh token

* Write a simple server application that listens on a port
* Use `ngrok` to tunnel requests to that port
* Set `ngrok` URL as the redirect URI on Spotify API Dashboard
* Use the code returned to get access and refresh tokens

In [78]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

class SpotifySearch:
    def __init__(self):
        self.spotipy = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
    
    def search(self, search_query):
        """
        Search through Spotify search API and return results
        """
        return self.spotipy.search(search_query)

    def get_first_track(self,search_query):
        results = self.search(search_query)
        res = {}
        if len(results) > 0:
            try:
                first = results['tracks']['items'][0]
                res['url'] = first['external_urls']['spotify']
                res['api_url'] = first['href']
                res['uri'] = first['uri']
            except KeyError as e:
                print(f'[error] URLs/URIs not found for {search_query}')
        return res


In [77]:
spotify = SpotifySearch()
spotify.get_first_track('bts stay gold')

{'url': 'https://open.spotify.com/track/0UwQTbq7ZtBJ7v530mpaxP',
 'api_url': 'https://api.spotify.com/v1/tracks/0UwQTbq7ZtBJ7v530mpaxP',
 'uri': 'spotify:track:0UwQTbq7ZtBJ7v530mpaxP'}

In [140]:
scope_bases, scopes = ['playlist-modify', 'playlist-read', 'library-read', 'library-modify'], []
for scope_base in scope_bases:
    scopes.extend([scope_base + '-public', scope_base + '-private'])
scope = ' '.join(scopes)

scope

'playlist-modify-public playlist-modify-private playlist-read-public playlist-read-private library-read-public library-read-private library-modify-public library-modify-private'

In [138]:
from spotipy.oauth2 import SpotifyOAuth
creds = SpotifyOAuth(scope=scope, username='aadithpm')

refresh_token = creds.get_access_token()['refresh_token']

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  after removing the cwd from sys.path.
